Eval modele
solution d'appel du retriver sur une query
sur la base des attendu


In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from rag_engine.llm_loader import load_llm
TOK, LLM = load_llm()

from pipeline import rag_pipeline, build_context, vector_store


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import json
from itertools import islice

# Fichiers dataset
chemin_queries = "dataset_eval/queries.json"
chemin_answers = "dataset_eval/answers.json"

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

queries = load_json(chemin_queries)
answers = load_json(chemin_answers)

# On prend les 5 premiers (clé, question)
first_five = list(islice(queries.items(), 5))

for key, query in first_five:
    print(f"❓ Question ({key}) : {query}\n")
    
    model_answer = rag_pipeline(query)
    
    print(f"🤖 Réponse LLM :\n{model_answer}\n")
    print(f"✅ Réponse attendue :\n{answers[key]}\n")
    print("────────────────────────────────────────────")


❓ Question (6ca4da60-a74c-45cb-b41c-c5fac380b7b9) : Quel est l'objectif de gestion du FCP décrit dans le document ?



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


🤖 Réponse LLM :
Answer: L'objectif de gestion du FCP est, sur la durée de placement recommandée supérieure à 5 ans, de surperformer l'indice ICE BofA Euro Corporate (ER00 Index), en s'exposant aux marchés internationaux de taux et de crédit. Cette gestion est mise en oeuvre de façon discrétionnaire au sein de l'OPCVM. Les décisions d'investissement intègrent à la fois des critères financiers et extra-financ

✅ Réponse attendue :
La recherche d'une performance sur la durée de placement recommandée supérieure à 5 ans par un placement sur les marchés financiers actions internationales, pays émergents inclus, tout en tenant compte des enjeux de développement durable.

────────────────────────────────────────────
❓ Question (0be2c984-debb-4c78-96f9-754dce277409) : Quels critères sont pris en compte dans les décisions d'investissement du FCP ?

🤖 Réponse LLM :
Answer: Les critères de responsabilité environnementale, sociale et de gouvernance de l'entreprise (ESG) sont pris en compte dans les

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


🤖 Réponse LLM :
Answer: La prise en compte des critères ESG (Environnement, Social, Gouvernance) dans la sélection de titres du FCP vise à évaluer la capacité des entreprises à transformer les enjeux du développement durable en vecteurs de performance.

✅ Réponse attendue :
Evaluer la capacité des entreprises à transformer les enjeux du développement durable en vecteurs de performance.

────────────────────────────────────────────
❓ Question (8a72a582-d7bf-44ad-84d5-fc3e4b8b8262) : Quels sont les scénarios mentionnés pour un investissement utilisant un mandataire approprié ?

🤖 Réponse LLM :
Answer: Scénario favorable : entre le 31/10/2016 et le 29/10/2021. Scénario intermédiaire : entre le 30/10/2015 et le 30/10/2020 ou entre le 28/02/2017 et le 28/02/2022. Scénario défavorable : entre le 31/12/2021 et le 15/1

✅ Réponse attendue :
Scénario favorable, scénario intermédiaire, scénario défavorable

────────────────────────────────────────────
❓ Question (e943d006-d450-4c3c-897a-4ca810a9

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


🤖 Réponse LLM :
Answer: Scénario favorable : entre le 31/10/2016 et le 29/10/2021
Scénario intermédiaire : entre le 30/10/2015 et le 30/10/2020
Scénario défavorable : entre le 31/12/2021 et le 15/12/2023

✅ Réponse attendue :
Scénario favorable : entre le 31/10/2016 et le 29/10/2021, Scénario intermédiaire : entre le 30/04/2014 et le 30/04/2019, Scénario défavorable : entre le 31/12/2021 et le 12/10/2023

────────────────────────────────────────────


In [5]:
import time

correct = 0
start = time.time()

for key, query in first_five:
    answer_gt = answers[key]
    answer_model = rag_pipeline(query)

    is_correct = answer_gt.lower() in answer_model.lower()
    correct += int(is_correct)

    print(f"❓ {query}")
    print(f"🤖 {answer_model}")
    print(f"✅ {answer_gt}")
    print(f"🎯 Match: {is_correct}")
    print("───────────────────")

print(f"Temps total: {time.time()-start:.2f}s")
print(f"Score: {correct}/{len(first_five)}")


AttributeError: 'Document' object has no attribute 'text'

In [3]:
try:
    import torch
    print(torch.cuda.memory_summary())
except:
    print("CPU mode")


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  16328 MiB |  16328 MiB |  43086 MiB |  26757 MiB |
|       from large pool |  16281 MiB |  16281 MiB |  42905 MiB |  26624 MiB |
|       from small pool |     47 MiB |     47 MiB |    181 MiB |    133 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  16328 MiB |  16328 MiB |  43086 MiB |  26757 MiB |
|       from large pool |  16281 MiB |  16281 MiB |  42905 MiB |

In [6]:
model_answer = rag_pipeline(query)

AttributeError: 'Document' object has no attribute 'text'